In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

In [2]:
import argparse 

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

In [3]:
import matplotlib.pyplot as plt
from pathlib import Path

def plot_history(history):
    log.info("History keys: %s", history.history.keys())
    # Accuracy
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(history.history['accuracy'], label='Train')
    ax.plot(history.history['val_accuracy'], label='Test')
    ax.set_title('Model accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.grid(True)
    ax.legend(['Train', 'Val'], loc='lower right')
    
    # Loss
    plt.figure(figsize=(10,5))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.grid()
    plt.show()
    
# couldn't make it work
# def build_dataset(data_folder):
#     def preprocess(line):
#         csv_records = tf.io.decode_csv(line, record_defaults=[tf.constant(0)]*4)
#         print(csv_records)
#         x = tf.stack(csv_records)
#         return x
#     dataset = tf.data.Dataset.list_files(data_folder + 'chorale_*.csv', seed=args.seed)
#     dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset = dataset.map(preprocess)
# #     dataset = dataset.shuffle(10000)
#     dataset = dataset.batch(args.batch_size).prefetch(1)
#     for d in dataset:
#         print(d)
#         break
#     return dataset

def build_dataset(data_folder):
    paths = Path(data_folder).glob('**/chorale_*.csv')
    np_list = [pd.read_csv(p).to_numpy() for p in paths]
    ragged = tf.ragged.constant(np_list)
    dataset = tf.data.Dataset.from_tensor_slices(ragged)
    def create_target(batch):
        print(batch)
        return batch
#     def batch_window(window):
#         return window.batch(32 + 1)
#     def to_windows(chorale):
#         dataset = tf.data.Dataset.from_tensor_slices(chorale)
#         dataset = dataset.window(32 + 1, 16, drop_remainder=True)
#         return dataset.flat_map(batch_window)
    dataset = dataset.flat_map(tf.data.Dataset.from_tensor_slices).flat_map(tf.data.Dataset.from_tensor_slices) # to a seq of notes
    dataset = dataset.batch(32)
    dataset = dataset.map(create_target)
    print(dataset)
    for d in dataset:
        print(d)
        break
#     def preprocess(line):
#         csv_records = tf.io.decode_csv(line, record_defaults=[tf.constant(0)]*4)
#         print(csv_records)
#         x = tf.stack(csv_records)
#         return x
#     dataset = tf.data.Dataset.list_files(data_folder + 'chorale_*.csv', seed=args.seed)
#     dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset = dataset.map(preprocess)
# #     dataset = dataset.shuffle(10000)
#     dataset = dataset.batch(args.batch_size).prefetch(1)

#     return dataset



def baseline_model():
    model = keras.models.Sequential()
    model.add(keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 4]))
    model.add(keras.layers.SimpleRNN(20))
    model.add(keras.layers.Dense(4))
    model.compile(loss='mse',optimizer=keras.optimizers.Nadam(learning_rate=0.01), metrics=[keras.metrics.mean_squared_error])
    return model

train_dataset = build_dataset(data_folder=args.train_folder)
# val_dataset = build_dataset(data_folder=args.val_folder)
# model = baseline_model()
# model.fit(train_dataset, epochs=1, validation_data=val_dataset)

Tensor("args_0:0", shape=(None,), dtype=int32)
<MapDataset shapes: (None,), types: tf.int32>
tf.Tensor(
[66 61 57 54 66 61 57 54 68 61 59 54 68 61 59 54 69 66 61 54 69 66 61 56
 69 66 61 57 69 66 61 59], shape=(32,), dtype=int32)


In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

import argparse 

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

from pathlib import Path

def create_target(batch):
    print(batch)
    X = batch[:, :-1]
    Y = batch[:, 1:] # predict next note in each arpegio, at each step
    return X, Y

def preprocess(window):
    window = tf.where(window == 0, window, window - min_note + 1) # shift values
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        return dataset.flat_map(batch_window)

    chorales = tf.ragged.constant(chorales)
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
#     dataset = dataset.flat_map(to_windows).map(preprocess)
#     if cache:
#         dataset = dataset.cache()
#     if shuffle_buffer_size:
#         dataset = dataset.shuffle(shuffle_buffer_size)
#     dataset = dataset.batch(batch_size)
#     dataset = dataset.map(create_target)
    for d in dataset:
        print(d)
        break
    return dataset.prefetch(1)

paths = Path(args.train_folder).glob('**/chorale_*.csv')
np_list = [pd.read_csv(p) for p in paths]
# print(np_list)
bach_dataset(np_list)

tf.Tensor(
[b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'note0' b'note1' b'note2' b'note3'
 b'note0' b'note1' b'note2' b'note3' b'n

<PrefetchDataset shapes: (None,), types: tf.string>

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
               [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
for d in dataset:
    print(d)
    break

dataset = dataset.flat_map(lambda x: tf.data.Dataset.from_tensor_slices(x))

for d in dataset:
    print(d)
    break